# 1 Introduction and clinical context

# 2 Defintions methods and hypotheses

# 3 Import of data and libraries

## 3.1 Import of libraries

In [6]:
#Importing basic data analyst libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Importing libraries for SQL querrying
import duckdb 

##3.2 Importing tables

I decided to import all the tables relevant for my analysis at this point. I will treat them as a database and later on i will look at them in detail. Now I will just import them one by one whereas querrying and cleaning comes a bit later in separate stages. I chose this approach because i find it more logically organised compared to other solutions.

###3.2.1 Importing confirmed cases table

In [10]:
df_cases = pd.read_csv('data/raw/confirmed_global.csv')

Verifying import

In [12]:
print(df_cases.head())

  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  4/1/21  4/2/21  4/3/21  4/4/21  \
0        0        0        0        0  ...   56517   56572   56595   56676   
1        0        0        0        0  ...  125506  125842  126183  126531   
2        0        0        0        0  ...  117304  117429  117524  117622   
3        0        0        0        0  ...   12053   12115   12174   12231   
4        0        0        0        0  ...   22399   22467   22579   22631   

   4/5/21  4/6/21  4/7/21  4/8/21  4/9/21  4/10/21  
0   56717   56779   56873   56943

##### ---Cases dataset imported successfully---

### 3.2.2 Importing death cases

In [15]:
df_deaths = pd.read_csv('data/raw/deaths_global.csv')

Verifying import

In [17]:
print(df_deaths.head())

  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  4/1/21  4/2/21  4/3/21  4/4/21  \
0        0        0        0        0  ...    2489    2495    2496    2497   
1        0        0        0        0  ...    2241    2247    2256    2265   
2        0        0        0        0  ...    3096    3099    3102    3105   
3        0        0        0        0  ...     115     116     117     117   
4        0        0        0        0  ...     538     538     540     542   

   4/5/21  4/6/21  4/7/21  4/8/21  4/9/21  4/10/21  
0    2508    2512    2512    2516

##### ----Deaths dataset importet successfully----

### 3.2.3 Importing recovered cases dataset

In [20]:
df_recovered = pd.read_csv('data/raw/recovered_global.csv')

In [21]:
print(df_recovered.head())

  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  4/1/21  4/2/21  4/3/21  4/4/21  \
0        0        0        0        0  ...   51788   51798   51802   51885   
1        0        0        0        0  ...   91875   92500   93173   93842   
2        0        0        0        0  ...   81632   81729   81813   81896   
3        0        0        0        0  ...   11365   11401   11428   11474   
4        0        0        0        0  ...   20508   20867   20871   20879   

   4/5/21  4/6/21  4/7/21  4/8/21  4/9/21  4/10/21  
0   51902   51928   51940   51956

##### ----Recovered dataset importet successfully!----

# 4 Inspecting cleaning and preparing datasets for melting

In this sections I will get acquianted with dataset, melt it, deal with missings and prepare it for melting

## 4.1 Inspecting cleaning and preparing the dataset with cases

### 4.1.1 Inspecting the dataset with cases

In [27]:
print(df_cases.head())

  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  4/1/21  4/2/21  4/3/21  4/4/21  \
0        0        0        0        0  ...   56517   56572   56595   56676   
1        0        0        0        0  ...  125506  125842  126183  126531   
2        0        0        0        0  ...  117304  117429  117524  117622   
3        0        0        0        0  ...   12053   12115   12174   12231   
4        0        0        0        0  ...   22399   22467   22579   22631   

   4/5/21  4/6/21  4/7/21  4/8/21  4/9/21  4/10/21  
0   56717   56779   56873   56943

In [28]:
print(df_cases.shape)

(274, 449)


Shape of the dataset is 274 rows and 449 columns. 

From the first glance I see identifier columns such as province/state, Country/Relogion, Latitude and longitude. There are 449 columns where majority corresponds to individual dates within given period of time. To confirm that only date columns follow 'Long', and that there are no unexpected columns in between, I will iterate through the column names and count the columns to crosscheck whether i itterate through columns correctly.

In [31]:
count = 0
for col in df_cases:
    print(col)
    count = count + 1
    
print("the column count is: ", count)

Province/State
Country/Region
Lat
Long
1/22/20
1/23/20
1/24/20
1/25/20
1/26/20
1/27/20
1/28/20
1/29/20
1/30/20
1/31/20
2/1/20
2/2/20
2/3/20
2/4/20
2/5/20
2/6/20
2/7/20
2/8/20
2/9/20
2/10/20
2/11/20
2/12/20
2/13/20
2/14/20
2/15/20
2/16/20
2/17/20
2/18/20
2/19/20
2/20/20
2/21/20
2/22/20
2/23/20
2/24/20
2/25/20
2/26/20
2/27/20
2/28/20
2/29/20
3/1/20
3/2/20
3/3/20
3/4/20
3/5/20
3/6/20
3/7/20
3/8/20
3/9/20
3/10/20
3/11/20
3/12/20
3/13/20
3/14/20
3/15/20
3/16/20
3/17/20
3/18/20
3/19/20
3/20/20
3/21/20
3/22/20
3/23/20
3/24/20
3/25/20
3/26/20
3/27/20
3/28/20
3/29/20
3/30/20
3/31/20
4/1/20
4/2/20
4/3/20
4/4/20
4/5/20
4/6/20
4/7/20
4/8/20
4/9/20
4/10/20
4/11/20
4/12/20
4/13/20
4/14/20
4/15/20
4/16/20
4/17/20
4/18/20
4/19/20
4/20/20
4/21/20
4/22/20
4/23/20
4/24/20
4/25/20
4/26/20
4/27/20
4/28/20
4/29/20
4/30/20
5/1/20
5/2/20
5/3/20
5/4/20
5/5/20
5/6/20
5/7/20
5/8/20
5/9/20
5/10/20
5/11/20
5/12/20
5/13/20
5/14/20
5/15/20
5/16/20
5/17/20
5/18/20
5/19/20
5/20/20
5/21/20
5/22/20
5/23/20
5/24/20
5/25/

All the respective columns  after 'Long' seem to represent individual dates and column count from the loop is in agreement with the column count from df_cases.head(). Next I need to check data types of individual columns

In [33]:
count_integer = 0
count_other_datatypes = 0
for col in df_cases:
    print(df_cases[col].dtype)
    if df_cases[col].dtype == int:
        count_integer = count_integer +1
    else:
        count_other_datatypes = count_other_datatypes + 1

print('Integer count is:', count_integer, '\n Other data types count is:', count_other_datatypes)
if count_integer == 445:
    print('All date-columns are integers!')

object
object
float64
float64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int6

Loop confirmed that all date columns are formated as an integer datatype. I see that other datatypes in the dataset are objects and float. I will confirm it with command .info()

In [35]:
df_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Columns: 449 entries, Province/State to 4/10/21
dtypes: float64(2), int64(445), object(2)
memory usage: 961.3+ KB


Verification showed exactly those datatypes i noticed  above

In [37]:
print(df_cases.describe)

<bound method NDFrame.describe of     Province/State      Country/Region        Lat        Long  1/22/20  \
0              NaN         Afghanistan  33.939110   67.709953        0   
1              NaN             Albania  41.153300   20.168300        0   
2              NaN             Algeria  28.033900    1.659600        0   
3              NaN             Andorra  42.506300    1.521800        0   
4              NaN              Angola -11.202700   17.873900        0   
..             ...                 ...        ...         ...      ...   
269            NaN             Vietnam  14.058324  108.277199        0   
270            NaN  West Bank and Gaza  31.952200   35.233200        0   
271            NaN               Yemen  15.552727   48.516388        0   
272            NaN              Zambia -13.133897   27.849332        0   
273            NaN            Zimbabwe -19.015438   29.154857        0   

     1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  ...  4/1/21  4/2/21  4/3/21

Describe command gives me an overview over the dataset. There are a lot of missing values in province/state. I have to investigate this variable closer. Country/region column will likely be the variable of interest and will serve us as identifier, Whereas latitute and longitude are not interesting forour analysis. Even though some viruses are known to spread better in colder or warmer climates, Cross-country comparisons in cases between the countries are not feasible because we don´t have data on people tested and countries were very different in test measures and  test equipment. In this dataset we lack the data for it. I will not check dataset with .describe(). Quick overview shows that there were 0 cases in a begining whereas number of cases began to increase and fluctuate over time.

### 4.1.2 Cleaning the dataset with cases

I am checking if there are any missing values in columns

In [41]:
df_cases.isna()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
270,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
271,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
272,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


I cannot see the whole dataset, but the output indicates that Province/state has missing values. I am going to investigate it further along with other columns

In [43]:
df_cases.isna().any()

Province/State     True
Country/Region    False
Lat                True
Long               True
1/22/20           False
                  ...  
4/6/21            False
4/7/21            False
4/8/21            False
4/9/21            False
4/10/21           False
Length: 449, dtype: bool

It seems that longitude latitude and province contain misssing values whereas dates seem to contain no missing value. I need to check my assumption.

In [45]:
df_cases.isna().any().sum()


3

There are three columns in the df_cases. They are Province/State, Lat, Long.

There are three columns in the df_cases. They are Province/State, Lat, Long.
Before I go further with cleaning and droping, want to check which column is the most pertinent one to become identifier column during melting. The two most valiable options are Province/state and Country/Region. I will extend the jupyter notebook output to contain up to 500 rows and columns which should be sufficient size to cover the entire df_cases dataframe.

In [48]:
#extending output for rows
pd.set_option('display.max_rows', 300)
#extending output for columns
pd.set_option('display.max_columns', 500)

In [49]:
print(df_cases[['Province/State', 'Country/Region']])

                                   Province/State  \
0                                             NaN   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   
5                                             NaN   
6                                             NaN   
7                                             NaN   
8                    Australian Capital Territory   
9                                 New South Wales   
10                             Northern Territory   
11                                     Queensland   
12                                South Australia   
13                                       Tasmania   
14                                       Victoria   
15                              Western Australia   
16                                            NaN   
17                                            

I confirm that column 'Province/State' is of no interest as it shows either overseas territories and provinces/states within a few countries.Contrary, Norway and Slovakia are both within Country/Region column which will become our identifier. Now I will strip the dataset from all the unneccesarry columns and rows.

In [51]:
df_cases_filtered = """
SELECT *
FROM df_cases
WHERE "Country/Region" = 'Norway' OR "Country/Region" = 'Slovakia';
"""
df_cases = duckdb.query(df_cases_filtered).to_df()

Checking whether i filtered rows correctly

In [53]:
print(df_cases.iloc[:, 1:])
print(type(df_cases), df_cases.shape)



  Country/Region     Lat     Long  1/22/20  1/23/20  1/24/20  1/25/20  \
0         Norway  60.472   8.4689        0        0        0        0   
1       Slovakia  48.669  19.6990        0        0        0        0   

   1/26/20  1/27/20  1/28/20  1/29/20  1/30/20  1/31/20  2/1/20  2/2/20  \
0        0        0        0        0        0        0       0       0   
1        0        0        0        0        0        0       0       0   

   2/3/20  2/4/20  2/5/20  2/6/20  2/7/20  2/8/20  2/9/20  2/10/20  2/11/20  \
0       0       0       0       0       0       0       0        0        0   
1       0       0       0       0       0       0       0        0        0   

   2/12/20  2/13/20  2/14/20  2/15/20  2/16/20  2/17/20  2/18/20  2/19/20  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   

   2/20/20  2/21/20  2/22/20  2/23/20  2/24/20  2/25/20  2/26/20  2/27/20  \
0      

I correctly filtered just relevant rows and transformed querried data into dataframe. There are two rows- Slovakia and Norway with number of columns(449) remained unchanged. Now I will proceed with dropping all the unneccessarry columns. namely: 
'Province/State', 
'Long', 
'Lat'

As mentioned earlier, variables lattitude and longitude are of  no value in our analysis, whereas province/state is variable indicating a region within larger countries.Countries of interest are slovakia and Norway and we have no addotional data on specific locations within those countries.

In [55]:
df_cases.drop(['Province/State', 'Long','Lat',], axis = 1, inplace = True)

Checking if there are some missing values in the dataset:
df_cases_filtered.isna()

In [57]:

print(df_cases,df_cases.shape)
print('', df_cases.isna().any().sum())

  Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  \
0         Norway        0        0        0        0        0        0   
1       Slovakia        0        0        0        0        0        0   

   1/28/20  1/29/20  1/30/20  1/31/20  2/1/20  2/2/20  2/3/20  2/4/20  2/5/20  \
0        0        0        0        0       0       0       0       0       0   
1        0        0        0        0       0       0       0       0       0   

   2/6/20  2/7/20  2/8/20  2/9/20  2/10/20  2/11/20  2/12/20  2/13/20  \
0       0       0       0       0        0        0        0        0   
1       0       0       0       0        0        0        0        0   

   2/14/20  2/15/20  2/16/20  2/17/20  2/18/20  2/19/20  2/20/20  2/21/20  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   

   2/22/20  2/23/20  2/24/20  2/25/20  2/26/20  2/27/20  2/28/20  2/29/20  \
0   

I correctly dropped all the unneccessarry rows and dataset is ready to melt. But before I will do it I will prepare datasets with deaths and recovered cases.

## 4.2.1 Inspecting cleaning and preparing the dataset with deaths

### 4.1.1 Inspecting the dataset with deaths

In [61]:
print(df_deaths.shape)

(274, 449)


The deaths dataset has the same number of rows(274) and columns (449) as the cases dataset. its a good indication because the process of cleaning checking and extracting of data will be very simmilar.

In [63]:
print(df_deaths.head())

  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  1/29/20  1/30/20  1/31/20  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   
3        0        0        0        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   

   2/1/20  2/2/20  2/3/20  2/4/20  2/5/20  2/6/20  2/7/20  2/8/20  2/9/20  \
0       0      

Again, earlier dates indicate zero dates whereas number of deaths towards the later period is higher and steadily increasing Whereas first five variables: 'Province/State' 'Country/Region 'Lat''Long' seem to be identical through all the datasets. I have to confirm this assumption

I am checking what datatypes exist in the dataset deaths

In [66]:
df_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Columns: 449 entries, Province/State to 4/10/21
dtypes: float64(2), int64(445), object(2)
memory usage: 961.3+ KB


df_deaths dataset seem to have identical types  two float columns (lat, long), two object columns (Province/State Country/Region) and 445 integer(date columns). I will verify it

In [68]:
print(df_deaths['Country/Region'].dtype)
print(df_deaths['Country/Region'].dtype)
print(df_deaths['Long'].dtype)
print(df_deaths['Lat'].dtype)

object
object
float64
float64


I confirm my assumption about float and object columns, I will check the date(integer columns as well)

In [70]:
#Filtering dataset so there are only dates
df_deaths_only_date = df_deaths.iloc[:, 4:]
#Creating loop to check which type the column is
date_counter = 0
for col in df_deaths_only_date:
    print(df_deaths_only_date[col].dtype)
    date_counter +=1 

print(date_counter)
print(df_deaths_only_date.info())

int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int6

I confirm that the dataset contains 445 integers! I am finished with scrutinizing the dataset with deaths and I move further to clean it.

In [72]:
### 4.1.3 Cleaning the dataset with deaths

I first check whether columns contain missing values

In [74]:
print(df_deaths.isna().any())
df_deaths.isna().any().sum()



Province/State     True
Country/Region    False
Lat                True
Long               True
1/22/20           False
                  ...  
4/6/21            False
4/7/21            False
4/8/21            False
4/9/21            False
4/10/21           False
Length: 449, dtype: bool


3

#I will check how many columns with missing values are present. if there are three we know they are those:
 Province/State      
 Lat       
 Long

In [76]:
df_deaths.isna().any().sum()

3

I confirm the assumption about the missing values. Now I check identifier columns

In [78]:
print(df_deaths['Province/State'],df_deaths['Country/Region'])

0                                               NaN
1                                               NaN
2                                               NaN
3                                               NaN
4                                               NaN
5                                               NaN
6                                               NaN
7                                               NaN
8                      Australian Capital Territory
9                                   New South Wales
10                               Northern Territory
11                                       Queensland
12                                  South Australia
13                                         Tasmania
14                                         Victoria
15                                Western Australia
16                                              NaN
17                                              NaN
18                                              NaN
19          

I will drop the Long, Lat and Province/State columns as they are of no interest for us:

In [80]:
df_deaths.drop(['Province/State', 'Long', 'Lat'], axis = 1, inplace = True)

Filtering rows with Slovakia and Norway and re-writing original dataset:

In [82]:
df_deaths.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21
0,Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,2,3,3,4,4,4,4,4,4,8,10,12,15,16,16,17,18,18,19,22,23,29,30,30,30,33,36,36,40,40,43,47,52,58,59,61,64,68,72,85,90,95,104,106,109,115,120,122,127,132,136,153,168,170,173,178,188,194,206,217,219,220,221,228,236,247,250,258,266,274,298,304,313,331,361,373,388,409,430,450,455,475,482,495,508,550,552,573,585,597,617,638,674,682,716

In [84]:
#row-filtering
df_deaths_filtered = """
SELECT * FROM df_deaths
WHERE "Country/Region" = 'Slovakia' OR "Country/Region" = 'Norway' """
#re-writing df_deaths
df_deaths = duckdb.query(df_deaths_filtered).to_df()

In [168]:

print(' The dataset has', df_deaths.isna().any().sum(), 'missings', df_deaths.shape, df_deaths.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Columns: 446 entries, Country/Region to 4/10/21
dtypes: int64(445), object(1)
memory usage: 7.1+ KB
 The dataset has 0 missings (2, 446) None


In [130]:
print(df_deaths)

  Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  \
0       Slovakia        0        0        0        0        0        0   
1         Norway        0        0        0        0        0        0   

   1/28/20  1/29/20  1/30/20  1/31/20  2/1/20  2/2/20  2/3/20  2/4/20  2/5/20  \
0        0        0        0        0       0       0       0       0       0   
1        0        0        0        0       0       0       0       0       0   

   2/6/20  2/7/20  2/8/20  2/9/20  2/10/20  2/11/20  2/12/20  2/13/20  \
0       0       0       0       0        0        0        0        0   
1       0       0       0       0        0        0        0        0   

   2/14/20  2/15/20  2/16/20  2/17/20  2/18/20  2/19/20  2/20/20  2/21/20  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   

   2/22/20  2/23/20  2/24/20  2/25/20  2/26/20  2/27/20  2/28/20  2/29/20  \
0   

Dataset is correctly cleaned and filtered with dates as integers, identifier column som object and zero missing values.